In [16]:
import pandas as pd
import numpy as np
import os

# Etapa 1 - Criação do dataframe com dados populacionais por ano dos municípios brasileiros para serviR de base de cáclulo das variáveis [QUANTIDADE DE PROCEDIMENTOS POR 10 MIL HAB] E [VALORES DE PROCEDIEMNTOS POR 10 MIL HAB]

In [17]:
# Criando função para concatenar poulação dos municípios por ano, entre 2010 e 2022, a partir dos dados em arquivo csv na pasta ./data/demografico/ para uso em cálculo posterior
def load_population_data(folder_path):
    dfs = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            year = int(filename.split('.')[0])
            df = pd.read_csv(os.path.join(folder_path, filename), sep=';', names=['codigo', 'pop'], dtype={'codigo': str})
            df.set_index('codigo', inplace=True)
            df = df[~df.index.duplicated(keep='first')] # remove possíveis linhas duplicadas
            df.rename(columns={'pop': str(year)}, inplace=True)
            dfs.append(df.iloc[1:, :]) # ignora a primeira linha
    merged_df = pd.concat(dfs, axis=1)
    merged_df.fillna(0, inplace=True) # preenche valores nulos com 0
    return merged_df



In [18]:
# Aplicando a função fornecendo a pasta com dados como parâmetro
df_pop = load_population_data('E:/OneDrive/Documentos/GitHub/iris/data/demografico')


In [19]:
# checando o dataframe e o resultado da função de concatenação de arquivos csv
df_pop

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
codigo,,,,,,,,,,,,,
1100015,24392,24228,24069,25728,25652,25578,25506,25437,23167,22945,22728,22516,22516
1100023,90353,91570,92747,101269,102860,104401,105896,107345,106168,107863,109523,111148,111148
1100031,6313,6221,6132,6495,6424,6355,6289,6224,5438,5312,5188,5067,5067
1100049,78574,78958,79330,85863,86556,87226,87877,88507,84813,85359,85893,86416,86416
1100056,17029,16939,16852,18041,18013,17986,17959,17934,16444,16323,16204,16088,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222005,12548,12644,12737,13227,13343,13456,13567,13675,13746,13863,13977,14088,14088
5222054,7371,7475,7576,7933,8053,8171,8286,8397,8611,8743,8873,9002,9002
5222203,4735,4846,4954,5246,5371,5495,5615,5731,6026,6171,6312,6451,6451


In [20]:
# Checando quantidade de municípios
len(df_pop)

5570

In [21]:
# checando tipo de variáveis resultante da função 
df_pop.dtypes

2010    object
2011    object
2012    object
2013    object
2014    object
2015    object
2016    object
2017    object
2018    object
2019    object
2020    object
2021    object
2022    object
dtype: object

In [23]:
# convertendo dados para tipo integer
df_pop = df_pop.astype(int)

In [26]:
# Checando valores iguais a zero em cada coluna
num_zeros_por_coluna = (df_pop == 0).sum()
print(num_zeros_por_coluna)
del num_zeros_por_coluna # exclui a var temp, após verificação

2010    0
2011    0
2012    0
2013    0
2014    0
2015    0
2016    0
2017    0
2018    0
2019    0
2020    0
2021    0
2022    0
dtype: int64


Nota importante: cinco municípios dos 5.570 existentes atualmente não constavam nas bases de dados do IBGE entre 2010 e 2012, provavelmente foram criados após essa data. Os municípios tem os seguintes códigos: 1504752, 4212650, 4220000, 4314548, 5006275. Os dados inexistenets foram preenchidos com cálculo de população decrescente de 2012, para 2011 e para 2010 com abse na diferença da população anual dos anos subsequentes. Desse modo a matriz não resultará em dados nulos quando for utilizada para cálculos com outras matrizes para obtenção de dados relacionados à população, como é o caso da quantidade de procedimentos por 10 mil habitantes. 


In [37]:
import requests
# obtendo dados dos municípios listados por meio da API do IBGE

def obter_dados_municipios(codigos):
    dados_municipios = []
    for codigo in codigos.split("|"):
        try:
            url = f'https://servicodados.ibge.gov.br/api/v1/localidades/municipios/{codigo}'
            response = requests.get(url)
            if response.status_code == 200:
                dados_municipio = response.json()
                nome_municipio = dados_municipio['nome']
                uf_municipio = dados_municipio['microrregiao']['mesorregiao']['UF']['sigla']
                dados_municipios.append({'Nome': nome_municipio, 'UF': uf_municipio})
        except requests.exceptions.RequestException as e:
            print(f"Erro ao obter dados do município {codigo}: {e}")
    return dados_municipios



In [38]:
# fornece parâmetros com os códigos dos municípios para obtenção do nome e UF
codigos = '1504752|4212650|4220000|4314548|5006275'
municipios = obter_dados_municipios(codigos)
for municipio in municipios:
    print(f"Nome: {municipio['Nome']}, UF: {municipio['UF']}")


Nome: Mojuí dos Campos, UF: PA
Nome: Pescaria Brava, UF: SC
Nome: Balneário Rincão, UF: SC
Nome: Pinto Bandeira, UF: RS
Nome: Paraíso das Águas, UF: MS


3 criados em 2013, sendo que Balneário Rincão e Pinto Bandeira foram emancipados antes de 2010. De toda forma os dados de população para os 5 municípios nos 3 anos foram projetados de maneira a não causar viés nos dados.

In [40]:
# mostando os dados dos municípios e as projeções resultantes para população nos anos de 2010 a 2012. 
df_pop.loc[['1504752', '4212650', '4220000', '4314548', '5006275']]


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
codigo,,,,,,,,,,,,,
1504752,14905,15014,15123,15232,15341,15446,15548,15646,15982,16084,16184,16282,16282
4212650,9465,9539,9613,9687,9761,9835,9908,9980,10022,10091,10159,10225,10225
4220000,11040,11236,11432,11628,11824,12018,12212,12403,12570,12760,12946,13129,13129
4314548,2324,2443,2562,2681,2800,2824,2847,2868,2968,3003,3036,3068,3068
5006275,4627,4732,4837,4942,5047,5150,5251,5350,5455,5555,5654,5751,5751


# Etapa 2 - Criação do dataframe com dados dos procedimentos para calcular matrizes de treinamento da rede neural autoencoder mlp e dos algoritmos de ML supervisionado e não supervisionado

In [42]:
df_sia = pd.read_csv('./data/SIA_procedimentos.csv', sep='|')
df_sia

,cnes,competencia,estabelecimento,razao_social,uf,codigo_municipio,municipio,co_procedimento,procedimento,qtde_procedimentos,valor_procedimentos
0,329,201105,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108006,ACOMPANHAMENTO INTENSIVO DE PACIENTE EM SAUDE ...,166,3004.60
1,329,201102,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108010,ACOMPANHAMENTO NAO INTENSIVO DE PACIENTE EM SA...,52,772.20
2,329,201111,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108010,ACOMPANHAMENTO NAO INTENSIVO DE PACIENTE EM SA...,30,445.50
3,329,201209,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108010,ACOMPANHAMENTO NAO INTENSIVO DE PACIENTE EM SA...,42,623.70
4,329,201104,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108012,ACOMPANHAMENTO SEMI-INTENSIVO DE PACIENTES EM ...,110,1749.00
...,...,...,...,...,...,...,...,...,...,...,...
7477277,9993827,202009,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60480001,"ENOXAPARINA SÓDICA 40 MG/0,4 ML SOLUÇÃO INJETÁ...",30,0.00
7477278,9993827,202110,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60483001,SACUBITRIL VALSARTANA SÓDICA HIDRATADA 50 MG (...,56,193.76
7477279,9993827,202208,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60483001,SACUBITRIL VALSARTANA SÓDICA HIDRATADA 50 MG (...,56,193.76
7477280,9993827,202207,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60483002,SACUBITRIL VALSARTANA SÓDICA HIDRATADA 100 MG ...,60,207.60


In [44]:
df_sia.rename(columns={'co_procedimento': 'codigo_procedimento'}, inplace=True)

In [43]:
df_sih = pd.read_csv('./data/SIH_procedimentos.csv', sep='|')
df_sih

,cnes,competencia,estabelecimento,razao_social,uf,codigo_municipio,municipio,codigo_procedimento,procedimento,qtde_procedimentos,valor_procedimentos
0,35,201002,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303080078,TRATAMENTO DE ESTREPTOCOCCIAS,11,340.62
1,35,201002,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303090316,TRATAMENTO DAS POLIARTROPATIAS INFLAMATORIAS,11,238.70
2,35,201003,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303020083,TRATAMENTO DE OUTRAS DOENÇAS DO SANGUE E DOS Ó...,24,388.37
3,35,201003,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,409060135,HISTERECTOMIA TOTAL,12,658.03
4,35,201004,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303070102,TRATAMENTO DE OUTRAS DOENCAS DO APARELHO DIGES...,15,380.58
...,...,...,...,...,...,...,...,...,...,...,...
28657071,9950931,202211,HOSPITAL MUNICIPAL INTEGRADO SANTO AMARO,PREFEITURA DO MUNICIPIO DE SAO PAULO,SP,355030,SAO PAULO,407040099,HERNIOPLASTIA INGUINAL (BILATERAL),27,5490.54
28657072,9950931,202211,HOSPITAL MUNICIPAL INTEGRADO SANTO AMARO,PREFEITURA DO MUNICIPIO DE SAO PAULO,SP,355030,SAO PAULO,407040102,HERNIOPLASTIA INGUINAL / CRURAL (UNILATERAL),51,10845.49
28657073,9994289,202105,POLICLINICA MUNICIPAL ESTEFANIA FARIAS,PREFEITURA MUNICIPAL DE SURUBIM,PE,261450,SURUBIM,303010223,TRATAMENTO DE INFECÇÃO PELO CORONAVIRUS – COVI...,44,33000.00
28657074,9997784,202107,HOSPITAL E MATERNIDADE NAILA GONCALO,MUNICIPIO DE BACABAEIRA,MA,210125,BACABEIRA,411010034,PARTO CESARIANO,74,4867.57


In [45]:
df_sia['fonte'] = 'sia'
df_sih['fonte'] = 'sih'

In [46]:
df_sia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7477282 entries, 0 to 7477281
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   cnes                 int64  
 1   competencia          int64  
 2   estabelecimento      object 
 3   razao_social         object 
 4   uf                   object 
 5   codigo_municipio     float64
 6   municipio            object 
 7   codigo_procedimento  int64  
 8   procedimento         object 
 9   qtde_procedimentos   int64  
 10  valor_procedimentos  float64
 11  fonte                object 
dtypes: float64(2), int64(4), object(6)
memory usage: 684.6+ MB


In [47]:
df_sih.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28657076 entries, 0 to 28657075
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   cnes                 int64  
 1   competencia          int64  
 2   estabelecimento      object 
 3   razao_social         object 
 4   uf                   object 
 5   codigo_municipio     int64  
 6   municipio            object 
 7   codigo_procedimento  int64  
 8   procedimento         object 
 9   qtde_procedimentos   int64  
 10  valor_procedimentos  float64
 11  fonte                object 
dtypes: float64(1), int64(5), object(6)
memory usage: 2.6+ GB


In [53]:
df_procedimentos = pd.concat([df_sia, df_sih])
df_procedimentos.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 36134358 entries, 0 to 28657075
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   cnes                 int64  
 1   competencia          int64  
 2   estabelecimento      object 
 3   razao_social         object 
 4   uf                   object 
 5   codigo_municipio     float64
 6   municipio            object 
 7   codigo_procedimento  int64  
 8   procedimento         object 
 9   qtde_procedimentos   int64  
 10  valor_procedimentos  float64
 11  fonte                object 
dtypes: float64(2), int64(4), object(6)
memory usage: 3.5+ GB


In [77]:
df_qtde_proc_cnes = df_procedimentos.pivot_table(index=['cnes', 'procedimento'], columns='competencia', values='qtde_procedimentos').reset_index()
df_qtde_proc_cnes.columns.name = None

In [79]:
df_qtde_proc_cnes

,cnes,procedimento,201001,201002,201003,201004,201005,201006,201007,201008,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,27,COLECISTECTOMIA,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,27,COLPOPERINEOPLASTIA ANTERIOR E POSTERIOR,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27,HERNIOPLASTIA INGUINAL / CRURAL (UNILATERAL),NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27,HERNIOPLASTIA UMBILICAL,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27,PARTO CESARIANO,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359536,9997784,SALPINGECTOMIA UNI / BILATERAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1359537,9997784,TRATAMENTO DA MIGRANEA COMPLICADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN
1359538,9997784,TRATAMENTO DA PIELONEFRITE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,8.0,12.0,NaN,28.0,42.0,36.0,NaN
1359539,9997784,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,5.0,9.0,NaN


In [80]:
df_qtde_proc_cnes.isna().sum().mean()

1114001.3164556962

In [81]:
df_qtde_proc_mun = df_procedimentos.pivot_table(index=['codigo_municipio', 'procedimento'], columns='competencia', values='qtde_procedimentos').reset_index()
df_qtde_proc_mun.columns.name = None

In [82]:
df_qtde_proc_mun

,codigo_municipio,procedimento,201001,201002,201003,201004,201005,201006,201007,201008,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001.0,ACOMPANHAMENTO SEMI-INTENSIVO DE PACIENTES EM ...,NaN,168.0,168.0,768.0,840.0,1140.0,1128.0,1272.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110001.0,AMPUTAÇÃO / DESARTICULAÇÃO DE DEDO,NaN,NaN,NaN,3.0,NaN,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110001.0,APENDICECTOMIA,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110001.0,COLECISTECTOMIA,NaN,NaN,NaN,3.0,3.0,6.0,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110001.0,COLPOPERINEOPLASTIA ANTERIOR E POSTERIOR,NaN,NaN,3.0,6.0,3.0,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920690,530180.0,URETROSTOMIA PERINEAL / CUTANEA / EXTERNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
920691,530180.0,URETROTOMIA INTERNA,5.0,NaN,NaN,NaN,NaN,6.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
920692,530180.0,VASECTOMIA,NaN,12.0,17.0,6.0,12.0,11.0,9.5,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
920693,530180.0,VITRECTOMIA ANTERIOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
df_qtde_proc_mun.isna().sum().mean()

731135.3417721519